In [26]:
from os.path import join, curdir
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import cv2

### Vector đặc trưng gương mặt

In [27]:
db_path = join(curdir,"Database", "Database.json")
with open(db_path, "r") as db:
    database = json.load(db)

In [28]:
# i=1
# for label, embedding in database.items():
#     plt.figure(i)
#     for ebd in embedding:
#         plt.plot(ebd)
#     i+=1


### Cân bằng sáng (Histogram Equalization)

In [ ]:
# Ngan, Phuoc, Thinh, Thanh
img_path = "../Sub-Module/FaceDetect/fd/Ngan_IMG_3246.JPG crop.png"
# img_path = "../Sub-Module/FaceDetect/fd/Phuoc_IMG_3263.JPG crop.png"
# img_path = "../Sub-Module/FaceDetect/fd/Thinh_IMG_3285.JPG crop.png"
# img_path = "../Sub-Module/FaceDetect/fd/Thanh_IMG_3280.JPG crop.png"

In [29]:
test_img = cv2.imread(img_path)
scale_percent = 40  # percent of original size
width = int(test_img.shape[1] * scale_percent / 100)
height = int(test_img.shape[0] * scale_percent / 100)
test_img = cv2.resize(test_img, (width, height))


yuv_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2YUV)
yuv_img[:, :, 0] = cv2.equalizeHist(yuv_img[:, :, 0])
equ = cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)

# Stacking images side-by-side
res = np.hstack((test_img, equ))

# Show image input vs output
cv2.imshow("", res)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Cân bằng sáng thích ứng (Adaptive Histogram Equalization)

In [30]:
test_img = cv2.imread(img_path)
scale_percent = 40  # percent of original size
width = int(test_img.shape[1] * scale_percent / 100)
height = int(test_img.shape[0] * scale_percent / 100)
test_img = cv2.resize(test_img, (width, height))

clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
yuv_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2YUV)
yuv_img[:, :, 0] = clahe.apply(yuv_img[:, :, 0])
equ = cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)

# Stacking images side-by-side
res = np.hstack((test_img, equ))

# Show image input vs output
cv2.imshow("", res)
cv2.waitKey(0)
cv2.destroyAllWindows()